In [4]:
import requests
import json
import pandas as pd

items

In [7]:
# excel読み込み
df = pd.read_excel('./R5.9セルメディ税制対象品目一覧.xlsx',sheet_name='database',skiprows=1)
df = df[['販売名','ＪＡＮコード','包装単位','発売元企業名','製造販売業者名','有効成分名','価格','セルフメディケーション税制対象']]
df = df[df['販売名']!='PRD_NAME']
import random
df['価格'] =[random.randint(7,30)*100 for _ in range(df.shape[0])]
df = df[df['ＪＡＮコード'].notnull()]
df['有効成分名'] = df['有効成分名'].str.replace('、',',')
df['セルフメディケーション税制対象'] = df['セルフメディケーション税制対象'].astype(str)
df = df.fillna('-')
df.head(5)


,販売名,ＪＡＮコード,包装単位,発売元企業名,製造販売業者名,有効成分名,価格,セルフメディケーション税制対象
1,アイクリスタ11,4981736222712,１５ｍＬ,サンドラッググループ,佐賀製薬株式会社,"クロルフェニラミン,テトラヒドロゾリン",1900,1
2,アイサット抗菌,4987103046301,１０ｍＬ,-,ゼリア新薬工業株式会社,クロルフェニラミン,800,1
3,アイサット抗菌,4987103051701,１２ｍＬ,-,ゼリア新薬工業株式会社,クロルフェニラミン,1200,1
4,アイスラブゲル,4987241105274,１００ｇ,ロート製薬株式会社,ロート製薬株式会社,サリチル酸グリコール,1000,1
5,アイダムALクール,4987154601641,１５ｍＬ,グレートアンドグランド,キョーリンリメディオ株式会社,"クロルフェニラミン,テトラヒドロゾリン",2100,1


In [11]:
df[df['ＪＡＮコード']=='4987300032008']

,販売名,ＪＡＮコード,包装単位,発売元企業名,製造販売業者名,有効成分名,価格,セルフメディケーション税制対象
3683,エスタックイブ,4987300032008,30錠,エスエス製薬株式会社,エスエス製薬株式会社,イブプロフェン,2600,1


In [60]:
# JANコード重複を排除 連番を振って一番初めのレコードだけ残す
df['seq_no'] = df.groupby('ＪＡＮコード').cumcount()
df = df[df['seq_no']==0]


In [74]:
url = 'http://127.0.0.1:8000/item/'

for i in range(len(df)):
  selected_dict = dict(df.iloc[i])
  body = {
    'prd_cd' : selected_dict['ＪＡＮコード'],
    'prd_name' : selected_dict['販売名'],
    'pac_unit' : selected_dict['包装単位'],
    'sale_company' : selected_dict['発売元企業名'],
    'mnf_company' : selected_dict['製造販売業者名'],
    'ingrdnt' : selected_dict['有効成分名'],
    'prd_price' : int(selected_dict['価格']),
    'smd_yn_fg' : selected_dict['セルフメディケーション税制対象']
  }

  res = requests.post(url,json.dumps(body))


In [6]:
# url = 'http://127.0.0.1:8000/item/'
# body = {
#   'prd_cd' : '4987241148820',
#   'prd_name' : 'メンソレータムアクネス25メディカルクリームc',
#   'pac_unit' : '16g',
#   'sale_company' : 'ロート製薬',
#   'mnf_company' : 'ロート製薬',
#   'ingrdnt' : 'イブプロフェンピコノール、イブプロピルメチルフェノール',
#   'prd_price' : int(1200),
#   'smd_yn_fg' : '1'
# }

# res = requests.post(url,json.dumps(body))
# print(res)

<Response [500]>


purchase <br>
trans, tran_details

In [82]:
items =  [
            {
                'prd_cd': '00001',
                'prd_name': 'Drug A',
                'prd_price':1000
            },
            {
                'prd_cd': '00002',
                'prd_name': 'Drug B',
                'prd_price':2000
            },
        ]

item_prices = [int(item['prd_price']) for item in items]
total_amt = sum(item_prices)


url = 'http://127.0.0.1:8000/purchase'
body = {
    # 'trd_id': 1,
    'mbr_id':1,
    'emp_cd':'1234',
    'store_cd':'C01',
    'pos_no': '001',
    'total_amt':total_amt,
    'ttl_amt_ex_tax':total_amt*0.1,
    'items':items
}

headers = {'Content-Type': 'application/json'}  # Content-Type ヘッダーを手動で設定

res = requests.post(url, json.dumps(body), headers=headers)
print(res)
print(res.json())

<Response [200]>
{'status': 'success', 'message': {'合計金額': 3000, '合計金額（税抜）': 300}}


members

In [3]:
import datetime
import requests

url = 'http://127.0.0.1:8000/member'
body = {
  'mbr_nm' : 'Ema',
  'birth_dt' : datetime.date(2024,1,1).isoformat(),
  'sex' : 'Female',
  'pref' : 'Tokyo'
}
res = requests.post(url, json=body)  # ここでjsonキーワード引数を使用する
print(res)
print(res.json())

<Response [200]>
{'mbr_nm': 'Ema', 'birth_dt': '2024-01-01', 'sex': 'Female', 'pref': 'Tokyo', 'mbr_id': 3}
